# Assignment-02, Probability Model A First Look: An Introduction of Language Model

## Assignment

1. Review the course online programming code; 
2. Review the main questions; 
3. Using wikipedia corpus to build a language model. 

## 1. Review the course online programming code. 

In [3]:
import pandas as pd
import re
import jieba

df = pd.read_csv('sqlResult_1558435.csv', encoding = 'gb18030')

def token(string):
    return ''.join(re.findall('[\w|\d]+',string))

all_articles=df['content'].tolist()

all_articles=[token(str(a)) for a in all_articles]

def cut(string): return list(jieba.cut(string))

text=''
for a in all_articles[:10000]:
    text += a

valid_tokens=[t for t in cut(text) if t != ' ' and t !='n']

from collections import Counter

words_count=Counter(valid_tokens)

words_count.most_common(10)

frequences_all=[f for w,f in words_count.most_common()]
frequences_sum=sum(frequences_all)

def get_prob(word):
    esp=1/frequences_sum
    if word in words_count:
        return words_count[word]/frequences_sum
    else:
        return esp
    
from functools import reduce

def product(numbers):
    return reduce(lambda n1,n2:n1*n2,numbers)

def language_model_one_gram(string):
    words=cut(string)
    return product([get_prob(w) for w in words])

all_2_grams_words=[''.join(valid_tokens[i:i+2]) for i in range(len(valid_tokens[:-2]))]

_2_gram_sum=len(all_2_grams_words)
_2_gram_counter = Counter(all_2_grams_words)

def get_combination_prob(w1, w2):
    if w1 + w2 in _2_gram_counter: return _2_gram_counter[w1+w2] / _2_gram_sum
    else:
        return 1 / _2_gram_sum

def get_prob_2_gram(w1, w2):
    return get_combination_prob(w1, w2) / get_prob(w1)

def langauge_model_of_2_gram(sentence):
    sentence_probability = 1
    
    words = cut(sentence)
    
    for i, word in enumerate(words):
        if i == 0: 
            prob = get_prob(word)
        else:
            previous = words[i-1]
            prob = get_prob_2_gram(previous, word)
        sentence_probability *= prob
    
    return sentence_probability

need_compared = [
    "今天晚上请你吃大餐，我们一起吃日料 明天晚上请你吃大餐，我们一起吃苹果",
    "真事一只好看的小猫 真是一只好看的小猫",
    "今晚我去吃火锅 今晚火锅去吃我",
    "洋葱奶昔来一杯 养乐多绿来一杯"
]

for s in need_compared:
    s1, s2 = s.split()
    p1, p2 = langauge_model_of_2_gram(s1), langauge_model_of_2_gram(s2)
    
    better = s1 if p1 > p2 else s2
    
    print('{} is more possible'.format(better))
    print('-'*4 + ' {} with probility {}'.format(s1, p1))
    print('-'*4 + ' {} with probility {}'.format(s2, p2))

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\ADMINI~1\AppData\Local\Temp\jieba.cache
Loading model cost 0.829 seconds.
Prefix dict has been built succesfully.


明天晚上请你吃大餐，我们一起吃苹果 is more possible
---- 今天晚上请你吃大餐，我们一起吃日料 with probility 9.123227846956153e-26
---- 明天晚上请你吃大餐，我们一起吃苹果 with probility 9.123227846956153e-26
真是一只好看的小猫 is more possible
---- 真事一只好看的小猫 with probility 3.379972340324493e-18
---- 真是一只好看的小猫 with probility 1.6933652226560576e-15
今晚我去吃火锅 is more possible
---- 今晚我去吃火锅 with probility 7.714164873681997e-13
---- 今晚火锅去吃我 with probility 1.07267876327285e-13
养乐多绿来一杯 is more possible
---- 洋葱奶昔来一杯 with probility 8.576046812079107e-11
---- 养乐多绿来一杯 with probility 2.7160325500196917e-07


## 2. Review the main points of this lesson. 

##### 1. How to Github and Why do we use Jupyter and Pycharm; 

Ans: Github是一个代码托管的网站，可以用来控制代码版本以及多人合作，使用git工具可以将Github上的代码下载到本地以及将本地代码上传至Github。
Jupyter是一个功能强大的文本编辑器，可以运行Python代码，还可以编辑复杂的数学公式、显示图片等，使用起来十分方便。
Pycharm是一个Python的IDE，可以加断点，查看变量的实时值，在调试代码时非常方便，此外还能分析每段代码性能等，在实际开发过程中都是很实用的功能。

##### 2. What's the Probability Model?

Ans:概率模型是基于概率论和数理统计建立起来的模型。

##### 3. Can you came up with some sceneraies at which we could use Probability Model?

Ans:拼音输入法

##### 4. Why do we use probability and what's the difficult points for programming based on parsing and pattern match? 


Ans:概率模型的泛化比模板匹配要好。模板匹配的难点在于自然语言很难用若干个模板来表示。

##### 5. What's the Language Model;

Ans:语言模型是用来计算一个单词列表是句子的概率的模型 

#####  6. Can you came up with some sceneraies at which we could use Language Model?

Ans:搜索引擎输入联想

##### 7. What's the 1-gram language model;

Ans:1-gram模型认为一个单词在一个句子中出现的概率与其他单词出现的概率无关。

##### 8. What's the disadvantages and advantages of 1-gram language model;

Ans:优点：简单，计算的参数少，只需要计算每个单词出现的概率即可，可以解决大部分的问题。
缺点：不考虑单词之间的关联关系，不够精确。

##### 9.  What't the 2-gram models; 

Ans: 2-gram模型认为一个单词在一个句子中出现的概率与他前一个单词相关。

##### 10. what's the web crawler, and can you implement a simple crawler? 

Ans:网络爬虫是一种按照一定的规则，自动地抓取万维网信息的程序或者脚本。可以用正则表达式实现一个简单的爬虫，在一个给定的网页中抓取有用的信息。

##### 11.  There may be some issues to make our crwaler programming difficult, what are these, and how do we solve them?

Ans:难点有：网站有反爬虫机制，有些网页上有脚本无法直接抓取内容等。解决方法有：模拟人浏览网页的操作，使用webkit浏览器内核获取网页内容等。

##### 12. What't the Regular Expression and how to use?

Ans:正则表达式是一个有特定规则的字符串，用于过滤满足某些特征的文本。可以借助Python语言中的re包使用正则表达式。

## 3. Using Wikipedia dataset to finish the language model. 

Step 1: You need to download the corpus from wikipedis:
> https://dumps.wikimedia.org/zhwiki/20190401/

Step 2: You may need the help of wiki-extractor:

> https://github.com/attardi/wikiextractor

Step 3: Using the technologies and methods to finish the language model; 
> 

Step 4: Try some interested sentence pairs, and check if your model could fit them

> 

Step 5: If we need to solve following problems, how can language model help us? 

+ Voice Recognization.
从每个识别出单词的若干个候选里分别取出一个组成一句话，概率最大的作为识别结果。
+ Sogou *pinyin* input.
每输入一个音节，将候选的字或词组按出现概率进行排序。
+ Auto correction in search engine. 
与输入法类似，计算所有候选组合的概率，若输入的查询关键字概率低于特定阈值，自动选择概率最大的候选组合。
+ Abnormal Detection.

In [ ]:
import os
import jieba
import re
from hanziconv import HanziConv
from collections import Counter

def token(string):
    return ''.join(re.findall('[\w|\d]+',string))

def cut(string): return list(jieba.cut(string))


def get_prob(word):
    esp = 1 / frequences_sum
    if word in words_count:
        return words_count[word] / frequences_sum
    else:
        return esp


from functools import reduce


def product(numbers):
    return reduce(lambda n1, n2: n1 * n2, numbers)


def language_model_one_gram(string):
    words = cut(string)
    return product([get_prob(w) for w in words])


def get_combination_prob(w1, w2):
    if w1 + w2 in _2_gram_counter:
        return _2_gram_counter[w1 + w2] / _2_gram_sum
    else:
        return 1 / _2_gram_sum

def get_prob_2_gram(w1, w2):
    return get_combination_prob(w1, w2) / get_prob(w1)

def langauge_model_of_2_gram(sentence):
    sentence_probability = 1

    words = cut(sentence)

    for i, word in enumerate(words):
        if i == 0:
            prob = get_prob(word)
        else:
            previous = words[i - 1]
            prob = get_prob_2_gram(previous, word)
        sentence_probability *= prob

    return sentence_probability




def traverse_path(dir):
    list = os.listdir(dir)  # 列出文件夹下所有的目录与文件
    for i in range(0,len(list)):
        path = os.path.join(dir,list[i])
        if os.path.isfile(path):
            global all_articles
            all_articles+=[HanziConv.toSimplified(r.strip()) for r in open(path,encoding='utf-8').readlines() if not r.startswith('<doc') and not r.startswith('</doc') and r!='\n']
        else:
            traverse_path(path)

rootdir='text\AA'

all_articles=[]
traverse_path(rootdir)
text=''
print(len(all_articles))
for a in all_articles[:100000]:
    text += a
print('2')
valid_tokens=[t for t in cut(text) if t != ' ' and t !='n']
words_count=Counter(valid_tokens)

frequences_all=[f for w,f in words_count.most_common()]
frequences_sum=sum(frequences_all)

all_2_grams_words=[''.join(valid_tokens[i:i+2]) for i in range(len(valid_tokens[:-2]))]

_2_gram_sum=len(all_2_grams_words)
_2_gram_counter = Counter(all_2_grams_words)


need_compared = [
    "今天晚上请你吃大餐，我们一起吃日料 明天晚上请你吃大餐，我们一起吃苹果",
    "真事一只好看的小猫 真是一只好看的小猫",
    "今晚我去吃火锅 今晚火锅去吃我",
    "洋葱奶昔来一杯 养乐多绿来一杯"
]

for s in need_compared:
    s1, s2 = s.split()
    p1, p2 = langauge_model_of_2_gram(s1), langauge_model_of_2_gram(s2)

    better = s1 if p1 > p2 else s2

    print('{} is more possible'.format(better))
    print('-' * 4 + ' {} with probility {}'.format(s1, p1))
    print('-' * 4 + ' {} with probility {}'.format(s2, p2))

### Compared to the previous learned parsing and pattern match problems. What's the advantage and disavantage of Probability Based Methods? 

Ans: 优点：泛化性更好，程序代码相对更简洁。缺点：正确率受语料特征影响较大。

## (Optional)  How to solve *OOV* problem?

If some words are not in our dictionary or corpus. When we using language model, we need to overcome this `out-of-vocabulary`(OOV) problems. There are so many intelligent man to solve this probelm. 

-- 

The first question is: 

**Q1: How did you solve this problem in your programming task?**

Ans: 未出现过的词按出现1次来计算概率。

Then, the sencond question is: 

**Q2: Read about the 'Turing-Good Estimator', can explain the main points about this method, and may implement this method in your programming task**

Reference: 
+ https://www.wikiwand.com/en/Good%E2%80%93Turing_frequency_estimation
+ https://github.com/Computing-Intelligence/References/blob/master/NLP/Natural-Language-Processing.pdf, Page-37

> coding in here